In [33]:
## NOTE: I used many ideas from this article and python notebook:
## https://towardsdatascience.com/machine-learning-with-pyspark-and-amazon-emr-3149dbc847ae
## https://github.com/maleckicoa/Sparkify-Project/blob/master/Sparkify_Big.ipynb

sc.install_pypi_package("pandas==0.25.1") 
sc.install_pypi_package("scikit-learn")
sc.install_pypi_package("quinn")

import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
spark = SparkSession \
    .builder \
    .appName("CS643_Project") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
## Load Training Dataset
train_df = spark.read.format('csv').options(header='true', inferSchema='true', sep=';').load('s3a://cs643johndaudelin/TrainingDataset.csv')
validation_df = spark.read.format('csv').options(header='true', inferSchema='true', sep=';').load('s3a://cs643johndaudelin/ValidationDataset.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
def remove_quotations(s):
    return s.replace('"', '')

train_df = quinn.with_columns_renamed(remove_quotations)(train_df)
train_df.withColumnRenamed('quality', 'label')

validation_df = quinn.with_columns_renamed(remove_quotations)(validation_df)
validation_df.withColumnRenamed('quality', 'label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
assembler = VectorAssembler(
    inputCols=["fixed acidity",
               "volatile acidity",
               "citric acid",
               "residual sugar",
               "chlorides",
               "free sulfur dioxide",
               "total sulfur dioxide",
               "density",
               "pH",
               "sulphates",
               "alcohol"],
                outputCol="inputFeatures")

scaler = Normalizer(inputCol="inputFeatures", outputCol="features")

lr=LogisticRegression()
rf= RandomForestClassifier()

pipeline1=Pipeline(stages=[assembler, scaler, lr])
pipeline2 = Pipeline(stages=[assembler, scaler, rf])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
paramgrid =ParamGridBuilder().build()

evaluator=MulticlassClassificationEvaluator(metricName="f1")

crossval= CrossValidator(estimator=pipeline1,  
                         estimatorParamMaps=paramgrid,
                         evaluator=evaluator, 
                         numFolds=3
                        )

cvModel1=crossval.fit(train_df) 
evaluator.evaluate(cvModel1.transform(validation_df))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.562631807944308

In [58]:
crossval= CrossValidator(estimator=pipeline2,  
                         estimatorParamMaps=paramgrid,
                         evaluator=evaluator, 
                         numFolds=3
                        )

cvModel2=crossval.fit(train_df) 
evaluator.evaluate(cvModel2.transform(validation_df))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.47683839637797576

In [ ]:
## Alternative way of making model: See https://towardsdatascience.com/your-first-apache-spark-ml-model-d2bb82b599dd

In [ ]:
sc.install_pypi_package("pyspark2pmml")

# from pyspark2pmml import PMMLBuilder
# pmmlBuilder = PMMLBuilder(sc, train_df, model).putOption(classifier, "compact", True)
# pmmlBuilder.buildFile("DecisionTreeModel.pmml")

## NEED TO INSTALL https://github.com/jpmml/jpmml-sparkml on EC2's

In [ ]:
from jpmml_sparkml import toPMMLBytes
pmmlBytes = toPMMLBytes(sc, df, pipelineModel)
print(pmmlBytes)